<a href="https://colab.research.google.com/github/tianchu-shu/DistinctiveSchools/blob/master/Chronic_Absenteeism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import matplotlib.pyplot as plt

In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls /content/gdrive/My\ Drive/Chronic_absenteeism_letter/

'Chronic Absence - Absence List Import.csv'     Chronic_Absenteeism.ipynb
'Chronic Absence Letter #1 - Jan '\''20.gdoc'  'CICS Prairie'
'Chronic Absence.xlsx'


# Load the sample Data

In [5]:
import pandas as pd
import glob, os

data_dir= '/content/gdrive/My Drive/Chronic_absenteeism_letter/'
filename = 'Chronic Absence.xlsx'
os.chdir(data_dir)

df=pd.read_excel(filename)

print("Column headings:")
print(df.columns)


Column headings:
Index(['NameOfInstitution', 'StudentUniqueID', 'LastSurname', 'FirstName',
       'GradeLevel', 'NumberOfSchoolTardies', 'NumberOfTotalAbsences',
       'Average Absence'],
      dtype='object')


In [0]:
dname=pd.read_excel(filename, sheet_name='Images  & PrincipalTable')

In [7]:
df.NameOfInstitution.unique()

array(['Prairie', 'Bucktown', 'Distinctive College Prep Harper Woods',
       'Distinctive College Prep Redford', 'West Belden', 'Irving Park'],
      dtype=object)

In [0]:
schoolname = {'Prairie':'CICS Prairie', 'Bucktown':'CICS Bucktown', 'Distinctive College Prep Harper Woods':'DCP Harper Woods',
       'Distinctive College Prep Redford':'DCP Redford', 'West Belden':'CICS West Belden', 'Irving Park':'CICS Irving Park'}

In [0]:
df['NameOfInstitution'] = df['NameOfInstitution'].map(schoolname)

In [10]:
df.head()

,NameOfInstitution,StudentUniqueID,LastSurname,FirstName,GradeLevel,NumberOfSchoolTardies,NumberOfTotalAbsences,Average Absence
0,CICS Prairie,50252756,Huizar Perez,Anthony,6,0,66,5
1,CICS Bucktown,50353442,Turner,Aiden,4,0,51,5
2,DCP Harper Woods,50000275,Smith,Paris,3,15,44,5
3,DCP Harper Woods,50000072,Phillips,Brandon,4,14,43,5
4,DCP Redford,2018249,Goforth,Brian,2,4,41,5


In [0]:
links = dict(zip(dname.Campus, dname.Link))

In [12]:
links

{'CICS Bucktown': 'https://www.cicsbucktown.org/uploads/8/4/9/1/84911918/published/cics-bucktown-horz.png?1531843975',
 'CICS Irving Park': 'https://www.cicsirvingpark.org/uploads/8/4/9/1/84911918/published/cics-irving-park-horz.png?1531844890',
 'CICS Prairie': 'https://www.cicsprairie.org/uploads/8/4/9/1/84911918/published/cics-prairie-horz.png?1531844962',
 'CICS West Belden': 'https://www.cicswestbelden.org/uploads/8/4/9/1/84911918/published/cics-west-belden-horz.png?1531845057',
 'DCP Harper Woods': 'https://www.dcpharperwoods.org/uploads/8/4/9/1/84911918/dscp-stacked-horizontal.png',
 'DCP Redford': 'https://www.dcpharperwoods.org/uploads/8/4/9/1/84911918/dscp-stacked-horizontal.png'}

In [0]:
#Make a dictionary matching the campus to Director names
dnames = dict(zip(dname.Campus, dname.Director))

In [14]:
dnames

{'CICS Bucktown': "Ms. Sarah O'Connell",
 'CICS Irving Park': 'Mr. JW Kuebler',
 'CICS Prairie': 'Ms. Jennifer Harth',
 'CICS West Belden': 'Ms. Colleen Collins',
 'DCP Harper Woods': 'Ms. Cassie Williams',
 'DCP Redford': 'Ms. Camille Hibbler'}

# Create the letter

In [15]:
!pip3 install reportlab

     |████████████████████████████████| 2.6MB 8.1MB/s 


In [0]:
import os

def createFolder(directory):

    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        



In [0]:
from reportlab.lib import colors
from reportlab.graphics.shapes import Drawing, String
from reportlab.graphics.charts.barcharts import HorizontalBarChart


def make_drawing(row):
    drawing = Drawing()
    data = [(row['NumberOfTotalAbsences'], row['Average Absence']),]
    names = [row['FirstName'], 'Classmates\n /Compañero']
    
    bc = HorizontalBarChart()
    bc.x = 20
    bc.y = 50
    bc.height = 150
    bc.width = 400
    bc.data = data
    bc.strokeColor = colors.black
    bc.valueAxis.valueMin = 0
    bc.valueAxis.visibleGrid = True
    bc.valueAxis.valueMax = 70
    bc.valueAxis.valueStep = 10

    bc.barLabels.fontName = "Helvetica"
    bc.barLabels.fontSize = 12
    bc.barLabelFormat = '%d'
    bc.barLabels.nudge = 7
    
    bc.categoryAxis.labels.boxAnchor = 'ne'
    bc.categoryAxis.labels.dx = -10
    bc.categoryAxis.labels.dy = 0 
    bc.categoryAxis.labels.fontName = 'Helvetica'
    bc.categoryAxis.categoryNames = names
    
    bc.valueAxis.visible = True
    #Customize the color of the bars  
    bc.bars[(0, 0)].fillColor = colors.steelblue 
    bc.bars[(0, 1)].fillColor = colors.lightsteelblue
    drawing.add(bc)

    return drawing

In [0]:
import datetime
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import PageBreak
from reportlab.lib.utils import ImageReader
 
def build_a_letter(row):
    '''
    Input: dataframe row
    Output: pdf file named after studentID
    '''
    #I name the pdf name after the StudentID
    name = str(row['StudentUniqueID'])+'.pdf'
    full_name = row['FirstName']+' ' + row['LastSurname']
    school_name = row.NameOfInstitution
    grade = str(row.GradeLevel)

    outfiledir = data_dir+str(school_name)+'/'+grade+'/'
    createFolder(outfiledir)
    outfilepath = os.path.join(outfiledir, name)
    #print(outfilepath)

    doc = SimpleDocTemplate(outfilepath, pagesize=letter,
                        rightMargin=72,leftMargin=72,
                        topMargin=72,bottomMargin=18)
    Story=[]

    if school_name == 'DCP Harper Woods' or school_name =='DCP Redford':
      logo = Image(links[school_name], width=2*inch, height=0.42*inch, kind='proportional')
    else:
      logo = Image(links[school_name])

    formatted_time = datetime.date.today().strftime("%B %d, %Y")
    NumberOfTotalAbsences = str(row.NumberOfTotalAbsences)
    story = []
    #Set the logo 
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=12>%s</font>' % formatted_time
     
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
     
    ptext = '<font size=12>Dear Family of %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    ptext = '<font size=12>We are writing to you because <u>our records show \
             that your child has missed more than 10 percent of the school year\
              to date.</u> Please understand that all absences are considered \
              unexcused until proper documentation is presented in accordance \
              with the school handbook.  Regular attendance at school is critical\
               for a child’s educational growth and achievement, and we are \
               committed to your child’s success.  We look forward to partnering\
                with you to ensure that your child continues to have a great \
                year of learning and growth.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=12>If absences continue, we will schedule a time for you\
     to come to school for a meeting with our administration, with the goal being\
      to identify the factors contributing to your child’s absenteeism, discuss \
      ways we can improve your child’s attendance this year, and commit to a plan\
       of action together. These steps will be taken in accordance with the \
       attendance policy for Distinctive Schools.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=12>Thank you for your attention to this matter, and \
    thank you in advance for your partnership and a dedication to ensuring that \
    your child is set up for success in school this year. I look forward to \
    working with you to support your child in having a positive and productive year. </font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    ptext = '<font size=12>Sincerely,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=12>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=12>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 
    
    #Second page spanish version
    Story.append(logo)
     
    styles=getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY))
    ptext = '<font size=12>%s</font>' % formatted_time
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=12><u><b>Primera notificación</b></u></font>' 
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=12>Dear Family of %s: </font>' % (full_name)
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext ='<font size=12>Le escribimos porque <u>nuestros registros muestran que \
    su hijo ha perdido más del 10 por ciento del año escolar hasta la fecha.</u> Por favor, \
    comprenda que todas las ausencias se consideran injustificadas hasta que se \
    presente la documentación adecuada de acuerdo con el manual de la escuela. \
    La asistencia regular a la escuela es fundamental para el crecimiento y el \
    rendimiento educativo de un niño, y estamos comprometidos con el éxito de su\
     hijo. Esperamos asociarnos con usted este año para asegurar que su hijo \
     continúe teniendo un gran año de aprendizaje y crecimiento.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))

    ptext = '<font size=12>Si las ausencias continúan, programaremos un horario \
    para que usted venga a la escuela a una reunión con nuestra administración \
    con el objetivo de identificar los factores que contribuyen al absentismo de\
     su hijo, discutir formas en que podemos mejorar la asistencia de su hijo \
     este año y comprometerse a un plan de acción juntos. Estos pasos se tomarán\
      de acuerdo con la política de asistencia de Distinctive Schools. </font>' 
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
     
    ptext = '<font size=12>Gracias por su atención a este asunto y gracias de \
    antemano por su colaboración y dedicación para garantizar que su hijo esté \
    preparado para el éxito en la escuela este año. Espero con interés trabajar \
    con usted para asegurarnos de que su hijo tenga un año positivo y productivo.</font>'
    Story.append(Paragraph(ptext, styles["Justify"]))
    Story.append(Spacer(1, 12))
    ptext = '<font size=12>Atentamente,</font>'
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(Spacer(1, 24))
    ptext = '<font size=12>%s</font>'% (dnames[school_name])
    Story.append(Paragraph(ptext, styles["Normal"]))
    ptext = '<font size=12>Director, %s</font>'% (school_name)
    Story.append(Paragraph(ptext, styles["Normal"]))
    Story.append(PageBreak()) 

    #Third page barchart
    title_style = styles['Heading1']
    title_style.alignment = 1
    title = Paragraph("Absences Summary/resumen de ausencias", title_style)
    Story.append(title)
    Story.append(Spacer(1, 24))
    pic = make_drawing(row)
    Story.append(pic)

    doc.build(Story)

In [0]:
for i in range(2):
  build_a_letter(df.loc[i])

# Bigquery

In [0]:
#

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [0]:
#Getting the original cohort's data

%%bigquery --project ds-student-data bdf
SELECT *
FROM `ds-student-data.Distinctive.ChronicAbsence` 


In [0]:
bdf.shape

In [0]:
bdf.head()

In [0]:
bdf['NumberOfTotalAbsences'].mean()

In [0]:
#Use the total number of absences for all schools for schools whose average is above all average
bdf['AverageAbsence'] = bdf['AverageAbsence'].apply(lambda x: 5.1 if x > 5.1 else x)

In [0]:
#Calculated the Peer Average Absences in the same school
#bdf['AverageAbsencePerSchool']=bdf.groupby(['NameOfInstitution'])['NumberOfTotalAbsences'].transform('mean')

In [0]:
x=bdf.loc[0,'NumberOfTotalAbsences':].plot.barh() 
